# step 5: analyze useful nouns

import librarys and load data set

In [1]:
# librarys
import pandas as pd
import nltk
import sklearn
import nltk.sentiment

import re
import pandas as pd
import nltk
import sklearn
from nltk.stem import WordNetLemmatizer
import numpy as np
from nltk.tokenize import word_tokenize
import nltk.sentiment
import string
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from tqdm import tqdm
from sklearn import linear_model
import collections
from gensim.models import Word2Vec
from gensim.models import word2vec
import multiprocessing
import time
import inflect
from textblob import TextBlob
import keras.preprocessing.text as T
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

## define stopwords
sr = stopwords.words('english')
p = inflect.engine()
wnl = WordNetLemmatizer() 
table = str.maketrans('', '', string.punctuation)

Using TensorFlow backend.


In [2]:
# function to change text to words
def clean_review(text):
    #Change n't into not
    x = re.sub(r'n\'t',' not',text)
    #Change not word into not_word
    x = re.sub(r'not ','not_',x)
    #Split into words
    x = word_tokenize(x)
    #Remove punctuation
    x = [w.translate(table) if not re.match(r'not_.*', w) else w for w in x]
    #Change numbers into words
    x = [p.number_to_words(w) if w.isdigit() else w  for w in x ]
    #Remove non alphabetic
    x = [w for w in x if w.isalpha() or re.match(r'not_.*',w)]
    #Convert to lower case
    x = [w.lower() for w in x]
    #Remove stop words
    x = [w for w in x if not w in sr]
    ## lemmatization
    x = [wnl.lemmatize(w) for w in x]
    return x

In [25]:
# load review data
review_bind=pd.read_csv("review_bind.csv")
review_bind.head()

,business_id,stars,bind_text
0,nsNONDHbV7Vudqh21uicqw,3.5,Was very excited for happy hour and heard grea...
1,F06m2yQSPHIrb1IT7heYeQ,4.0,A Cantonese restaurant on the southwest side o...
2,c_FTil8s5PS2l_YJDQAXhA,4.0,"Great food, friendly service, lovely atmospher..."
3,vAbRNF0hJt91oCUpm_sdKw,3.5,I'm a FAN. Loved this place. Great service a...
4,z8H4Mdzl4jS9pYswj6Jf9w,3.5,Service was really poor.\r\r\nMay try the King...


In [29]:
# load dictionary of adjectives with positive/negative tags
f = open('dict_adj.txt','r')
a = f.read()
dict_adj = eval(a)
f.close()

do word tokenization

In [5]:
# do word tokenization
clean_text = [[1]]*len(review_bind)
for i in range(len(review_bind)):
    clean_text[i]=clean_review(review_bind['bind_text'][i])

define findadj function to find adjectives near a target noun

In [9]:
# define findadj funciton
def findadj(noun,text):
    adj_word = [[1]]*len(text)
    for i in range(len(text)):
        word=text[i]        
        pos_tag = nltk.pos_tag(word)
        adjs = []
        for j in range(len(word)):
            if (word[j] == noun):
                if ((j-1)>=0 and pos_tag[j-1][1] == 'JJ'):
                    adjs.append(pos_tag[j-1][0])
        adj_word[i]=adjs
    return adj_word

## lobster

In [226]:
adj_lobster=findadj('lobster',clean_text)

In [38]:
lobster=pd.DataFrame(review_bind['business_id'])
lobster['positive']=[0]*(len(review_bind))
lobster['negative']=[0]*len(review_bind)
lobster['proportion']=[np.nan]*len(review_bind)
lobster.head()

,business_id,positive,negative,proportion
0,nsNONDHbV7Vudqh21uicqw,0,0,NaN
1,F06m2yQSPHIrb1IT7heYeQ,0,0,NaN
2,c_FTil8s5PS2l_YJDQAXhA,0,0,NaN
3,vAbRNF0hJt91oCUpm_sdKw,0,0,NaN
4,z8H4Mdzl4jS9pYswj6Jf9w,0,0,NaN


In [39]:
from collections import Counter

for i in range(len(adj_lobster)):
    if(len(adj_lobster[i]) != 0):
        dictionary=Counter(adj_lobster[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    lobster.iat[i,1]=lobster['positive'][i]+dictionary[word]
                else:
                    lobster.iat[i,2]=lobster['negative'][i]+dictionary[word]
        lobster.iat[i,3]=lobster['positive'][i]/(lobster['positive'][i]+lobster['negative'][i])
lobster.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in longlong_scalars
  if sys.path[0] == '':


,business_id,positive,negative,proportion
0,nsNONDHbV7Vudqh21uicqw,88,6,0.93617
1,F06m2yQSPHIrb1IT7heYeQ,3,0,1.00000
2,c_FTil8s5PS2l_YJDQAXhA,1,0,1.00000
3,vAbRNF0hJt91oCUpm_sdKw,15,1,0.93750
4,z8H4Mdzl4jS9pYswj6Jf9w,1,0,1.00000


In [41]:
np.sum(lobster['positive'])

6022

In [42]:
np.sum(lobster['negative'])

374

In [46]:
lobster['stars']=review_bind['stars']
lobster.head()

,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,88,6,0.93617,3.5
1,F06m2yQSPHIrb1IT7heYeQ,3,0,1.00000,4.0
2,c_FTil8s5PS2l_YJDQAXhA,1,0,1.00000,4.0
3,vAbRNF0hJt91oCUpm_sdKw,15,1,0.93750,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,1,0,1.00000,3.5


In [52]:
lobster.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.144111
stars,0.144111,1.000000


In [53]:
lobster.to_csv("lobster.csv",index=False)

# other nouns

In [54]:
noun_list=["lobster","crab","shrimp","oyster","fish","clam","sauce","wine","stuff","waiter","waitress","chef","manager","location","style","flavor","plate","service","price"]


## crab

In [225]:
adj_crab=findadj('crab',clean_text)

In [56]:
crab=pd.DataFrame(review_bind['business_id'])
crab['positive']=[0]*(len(review_bind))
crab['negative']=[0]*len(review_bind)
crab['proportion']=[np.nan]*len(review_bind)

In [57]:
from collections import Counter

for i in range(len(adj_crab)):
    if(len(adj_crab[i]) != 0):
        dictionary=Counter(adj_crab[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    crab.iat[i,1]=crab['positive'][i]+dictionary[word]
                else:
                    crab.iat[i,2]=crab['negative'][i]+dictionary[word]
        crab.iat[i,3]=crab['positive'][i]/(crab['positive'][i]+crab['negative'][i])

crab['stars']=review_bind['stars']
crab.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in longlong_scalars
  if sys.path[0] == '':


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,57,4,0.934426,3.5
1,F06m2yQSPHIrb1IT7heYeQ,3,1,0.750000,4.0
2,c_FTil8s5PS2l_YJDQAXhA,0,0,NaN,4.0
3,vAbRNF0hJt91oCUpm_sdKw,3,1,0.750000,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,0,0,NaN,3.5


In [58]:
print(np.sum(crab['positive']))
print(np.sum(crab['negative']))

3146
479


In [59]:
crab.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.141837
stars,0.141837,1.000000


In [60]:
crab.to_csv("crab.csv",index=False)

## shrimp

In [61]:
adj_shrimp=findadj('shrimp',clean_text)

In [63]:
shrimp=pd.DataFrame(review_bind['business_id'])
shrimp['positive']=[0]*(len(review_bind))
shrimp['negative']=[0]*len(review_bind)
shrimp['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_shrimp)):
    if(len(adj_shrimp[i]) != 0):
        dictionary=Counter(adj_shrimp[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    shrimp.iat[i,1]=shrimp['positive'][i]+dictionary[word]
                else:
                    shrimp.iat[i,2]=shrimp['negative'][i]+dictionary[word]
        shrimp.iat[i,3]=shrimp['positive'][i]/(shrimp['positive'][i]+shrimp['negative'][i])

shrimp['stars']=review_bind['stars']
shrimp.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,42,2,0.954545,3.5
1,F06m2yQSPHIrb1IT7heYeQ,0,0,NaN,4.0
2,c_FTil8s5PS2l_YJDQAXhA,0,0,NaN,4.0
3,vAbRNF0hJt91oCUpm_sdKw,2,0,1.000000,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,0,0,NaN,3.5


In [64]:
print(np.sum(shrimp['positive']))
print(np.sum(shrimp['negative']))

4379
269


In [65]:
shrimp.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.040376
stars,0.040376,1.000000


In [66]:
shrimp.to_csv("shrimp.csv",index=False)

## oyster

In [67]:
adj_oyster=findadj('oyster',clean_text)

In [69]:
oyster=pd.DataFrame(review_bind['business_id'])
oyster['positive']=[0]*(len(review_bind))
oyster['negative']=[0]*len(review_bind)
oyster['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_oyster)):
    if(len(adj_oyster[i]) != 0):
        dictionary=Counter(adj_oyster[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    oyster.iat[i,1]=oyster['positive'][i]+dictionary[word]
                else:
                    oyster.iat[i,2]=oyster['negative'][i]+dictionary[word]
        oyster.iat[i,3]=oyster['positive'][i]/(oyster['positive'][i]+oyster['negative'][i])

oyster['stars']=review_bind['stars']
oyster.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,41,2,0.953488,3.5
1,F06m2yQSPHIrb1IT7heYeQ,0,0,NaN,4.0
2,c_FTil8s5PS2l_YJDQAXhA,9,1,0.900000,4.0
3,vAbRNF0hJt91oCUpm_sdKw,1,0,1.000000,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,4,1,0.800000,3.5


In [70]:
print(np.sum(oyster['positive']))
print(np.sum(oyster['negative']))

3305
180


In [71]:
oyster.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.032166
stars,0.032166,1.000000


In [72]:
oyster.to_csv("oyster.csv",index=False)

## fish

In [73]:
adj_fish=findadj('fish',clean_text)

In [75]:
fish=pd.DataFrame(review_bind['business_id'])
fish['positive']=[0]*(len(review_bind))
fish['negative']=[0]*len(review_bind)
fish['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_fish)):
    if(len(adj_fish[i]) != 0):
        dictionary=Counter(adj_fish[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    fish.iat[i,1]=fish['positive'][i]+dictionary[word]
                else:
                    fish.iat[i,2]=fish['negative'][i]+dictionary[word]
        fish.iat[i,3]=fish['positive'][i]/(fish['positive'][i]+fish['negative'][i])

fish['stars']=review_bind['stars']
fish.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,32,3,0.914286,3.5
1,F06m2yQSPHIrb1IT7heYeQ,7,0,1.000000,4.0
2,c_FTil8s5PS2l_YJDQAXhA,3,0,1.000000,4.0
3,vAbRNF0hJt91oCUpm_sdKw,10,0,1.000000,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,19,0,1.000000,3.5


In [76]:
print(np.sum(fish['positive']))
print(np.sum(fish['negative']))

6646
558


In [77]:
fish.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.269021
stars,0.269021,1.000000


In [78]:
fish.to_csv("fish.csv",index=False)

## clam

In [79]:
adj_clam=findadj('clam',clean_text)

In [81]:
clam=pd.DataFrame(review_bind['business_id'])
clam['positive']=[0]*(len(review_bind))
clam['negative']=[0]*len(review_bind)
clam['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_clam)):
    if(len(adj_clam[i]) != 0):
        dictionary=Counter(adj_clam[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    clam.iat[i,1]=clam['positive'][i]+dictionary[word]
                else:
                    clam.iat[i,2]=clam['negative'][i]+dictionary[word]
        clam.iat[i,3]=clam['positive'][i]/(clam['positive'][i]+clam['negative'][i])

clam['stars']=review_bind['stars']
clam.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,20,2,0.909091,3.5
1,F06m2yQSPHIrb1IT7heYeQ,4,0,1.000000,4.0
2,c_FTil8s5PS2l_YJDQAXhA,0,0,NaN,4.0
3,vAbRNF0hJt91oCUpm_sdKw,9,0,1.000000,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,0,0,NaN,3.5


In [82]:
print(np.sum(clam['positive']))
print(np.sum(clam['negative']))

1154
100


In [83]:
clam.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.110865
stars,0.110865,1.000000


In [84]:
clam.to_csv("clam.csv",index=False)

## sauce

In [85]:
adj_sauce=findadj('sauce',clean_text)

In [87]:
sauce=pd.DataFrame(review_bind['business_id'])
sauce['positive']=[0]*(len(review_bind))
sauce['negative']=[0]*len(review_bind)
sauce['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_sauce)):
    if(len(adj_sauce[i]) != 0):
        dictionary=Counter(adj_sauce[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    sauce.iat[i,1]=sauce['positive'][i]+dictionary[word]
                else:
                    sauce.iat[i,2]=sauce['negative'][i]+dictionary[word]
        sauce.iat[i,3]=sauce['positive'][i]/(sauce['positive'][i]+sauce['negative'][i])

sauce['stars']=review_bind['stars']
sauce.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,18,3,0.857143,3.5
1,F06m2yQSPHIrb1IT7heYeQ,8,0,1.000000,4.0
2,c_FTil8s5PS2l_YJDQAXhA,2,0,1.000000,4.0
3,vAbRNF0hJt91oCUpm_sdKw,0,2,0.000000,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,4,0,1.000000,3.5


In [88]:
print(np.sum(sauce['positive']))
print(np.sum(sauce['negative']))

4909
209


In [89]:
sauce.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.127738
stars,0.127738,1.000000


In [90]:
sauce.to_csv("sauce.csv",index=False)

## wine

In [91]:
adj_wine=findadj('wine',clean_text)

In [93]:
wine=pd.DataFrame(review_bind['business_id'])
wine['positive']=[0]*(len(review_bind))
wine['negative']=[0]*len(review_bind)
wine['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_wine)):
    if(len(adj_wine[i]) != 0):
        dictionary=Counter(adj_wine[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    wine.iat[i,1]=wine['positive'][i]+dictionary[word]
                else:
                    wine.iat[i,2]=wine['negative'][i]+dictionary[word]
        wine.iat[i,3]=wine['positive'][i]/(wine['positive'][i]+wine['negative'][i])

wine['stars']=review_bind['stars']
wine.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,22,3,0.880000,3.5
1,F06m2yQSPHIrb1IT7heYeQ,0,0,NaN,4.0
2,c_FTil8s5PS2l_YJDQAXhA,3,0,1.000000,4.0
3,vAbRNF0hJt91oCUpm_sdKw,4,0,1.000000,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,23,1,0.958333,3.5


In [94]:
print(np.sum(wine['positive']))
print(np.sum(wine['negative']))

2170
119


In [95]:
wine.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,-0.059381
stars,-0.059381,1.000000


In [96]:
wine.to_csv("wine.csv",index=False)

## stuff

In [97]:
adj_stuff=findadj('stuff',clean_text)

In [99]:
stuff=pd.DataFrame(review_bind['business_id'])
stuff['positive']=[0]*(len(review_bind))
stuff['negative']=[0]*len(review_bind)
stuff['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_stuff)):
    if(len(adj_stuff[i]) != 0):
        dictionary=Counter(adj_stuff[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    stuff.iat[i,1]=stuff['positive'][i]+dictionary[word]
                else:
                    stuff.iat[i,2]=stuff['negative'][i]+dictionary[word]
        stuff.iat[i,3]=stuff['positive'][i]/(stuff['positive'][i]+stuff['negative'][i])

stuff['stars']=review_bind['stars']
stuff.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,5,0,1.0,3.5
1,F06m2yQSPHIrb1IT7heYeQ,0,0,NaN,4.0
2,c_FTil8s5PS2l_YJDQAXhA,0,0,NaN,4.0
3,vAbRNF0hJt91oCUpm_sdKw,0,0,NaN,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,0,0,NaN,3.5


In [100]:
print(np.sum(stuff['positive']))
print(np.sum(stuff['negative']))

802
58


In [101]:
stuff.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.055047
stars,0.055047,1.000000


In [102]:
stuff.to_csv("oyster.csv",index=False)

## waiter

In [103]:
adj_waiter=findadj('waiter',clean_text)

In [105]:
waiter=pd.DataFrame(review_bind['business_id'])
waiter['positive']=[0]*(len(review_bind))
waiter['negative']=[0]*len(review_bind)
waiter['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_waiter)):
    if(len(adj_waiter[i]) != 0):
        dictionary=Counter(adj_waiter[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    waiter.iat[i,1]=waiter['positive'][i]+dictionary[word]
                else:
                    waiter.iat[i,2]=waiter['negative'][i]+dictionary[word]
        waiter.iat[i,3]=waiter['positive'][i]/(waiter['positive'][i]+waiter['negative'][i])

waiter['stars']=review_bind['stars']
waiter.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,12,1,0.923077,3.5
1,F06m2yQSPHIrb1IT7heYeQ,1,0,1.000000,4.0
2,c_FTil8s5PS2l_YJDQAXhA,1,0,1.000000,4.0
3,vAbRNF0hJt91oCUpm_sdKw,7,3,0.700000,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,2,2,0.500000,3.5


In [106]:
print(np.sum(waiter['positive']))
print(np.sum(waiter['negative']))

1468
267


In [107]:
waiter.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.267945
stars,0.267945,1.000000


In [108]:
waiter.to_csv("waiter.csv",index=False)

## waitress

In [109]:
adj_waitress=findadj('waitress',clean_text)

In [111]:
waitress=pd.DataFrame(review_bind['business_id'])
waitress['positive']=[0]*(len(review_bind))
waitress['negative']=[0]*len(review_bind)
waitress['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_waitress)):
    if(len(adj_waitress[i]) != 0):
        dictionary=Counter(adj_waitress[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    waitress.iat[i,1]=waitress['positive'][i]+dictionary[word]
                else:
                    waitress.iat[i,2]=waitress['negative'][i]+dictionary[word]
        waitress.iat[i,3]=waitress['positive'][i]/(waitress['positive'][i]+waitress['negative'][i])

waitress['stars']=review_bind['stars']
waitress.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,9,2,0.818182,3.5
1,F06m2yQSPHIrb1IT7heYeQ,3,1,0.750000,4.0
2,c_FTil8s5PS2l_YJDQAXhA,1,0,1.000000,4.0
3,vAbRNF0hJt91oCUpm_sdKw,1,0,1.000000,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,4,0,1.000000,3.5


In [112]:
print(np.sum(waitress['positive']))
print(np.sum(waitress['negative']))

1424
284


In [113]:
waitress.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.164274
stars,0.164274,1.000000


In [114]:
waitress.to_csv("waitress.csv",index=False)

## chef

In [115]:
adj_chef=findadj('chef',clean_text)

In [117]:
chef=pd.DataFrame(review_bind['business_id'])
chef['positive']=[0]*(len(review_bind))
chef['negative']=[0]*len(review_bind)
chef['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_chef)):
    if(len(adj_chef[i]) != 0):
        dictionary=Counter(adj_chef[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    chef.iat[i,1]=chef['positive'][i]+dictionary[word]
                else:
                    chef.iat[i,2]=chef['negative'][i]+dictionary[word]
        chef.iat[i,3]=chef['positive'][i]/(chef['positive'][i]+chef['negative'][i])

chef['stars']=review_bind['stars']
chef.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,2,0,1.0,3.5
1,F06m2yQSPHIrb1IT7heYeQ,1,0,1.0,4.0
2,c_FTil8s5PS2l_YJDQAXhA,0,0,NaN,4.0
3,vAbRNF0hJt91oCUpm_sdKw,1,0,1.0,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,3,0,1.0,3.5


In [118]:
print(np.sum(chef['positive']))
print(np.sum(chef['negative']))

818
48


In [119]:
chef.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.252342
stars,0.252342,1.000000


In [120]:
chef.to_csv("chef.csv",index=False)

## manager

In [121]:
adj_manager=findadj('manager',clean_text)

In [123]:
manager=pd.DataFrame(review_bind['business_id'])
manager['positive']=[0]*(len(review_bind))
manager['negative']=[0]*len(review_bind)
manager['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_manager)):
    if(len(adj_manager[i]) != 0):
        dictionary=Counter(adj_manager[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    manager.iat[i,1]=manager['positive'][i]+dictionary[word]
                else:
                    manager.iat[i,2]=manager['negative'][i]+dictionary[word]
        manager.iat[i,3]=manager['positive'][i]/(manager['positive'][i]+manager['negative'][i])

manager['stars']=review_bind['stars']
manager.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,2,2,0.5,3.5
1,F06m2yQSPHIrb1IT7heYeQ,0,0,NaN,4.0
2,c_FTil8s5PS2l_YJDQAXhA,1,0,1.0,4.0
3,vAbRNF0hJt91oCUpm_sdKw,0,0,NaN,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,1,0,1.0,3.5


In [124]:
print(np.sum(manager['positive']))
print(np.sum(manager['negative']))

586
263


In [125]:
manager.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.209061
stars,0.209061,1.000000


In [126]:
manager.to_csv("manager.csv",index=False)

## location

In [127]:
adj_location=findadj('location',clean_text)

In [129]:
location=pd.DataFrame(review_bind['business_id'])
location['positive']=[0]*(len(review_bind))
location['negative']=[0]*len(review_bind)
location['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_location)):
    if(len(adj_location[i]) != 0):
        dictionary=Counter(adj_location[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    location.iat[i,1]=location['positive'][i]+dictionary[word]
                else:
                    location.iat[i,2]=location['negative'][i]+dictionary[word]
        location.iat[i,3]=location['positive'][i]/(location['positive'][i]+location['negative'][i])

location['stars']=review_bind['stars']
location.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,5,1,0.833333,3.5
1,F06m2yQSPHIrb1IT7heYeQ,0,0,NaN,4.0
2,c_FTil8s5PS2l_YJDQAXhA,2,0,1.000000,4.0
3,vAbRNF0hJt91oCUpm_sdKw,2,0,1.000000,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,10,0,1.000000,3.5


In [130]:
print(np.sum(location['positive']))
print(np.sum(location['negative']))

2053
99


In [131]:
location.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.058156
stars,0.058156,1.000000


In [132]:
location.to_csv("location.csv",index=False)

## style

In [133]:
adj_style=findadj('style',clean_text)

In [135]:
style=pd.DataFrame(review_bind['business_id'])
style['positive']=[0]*(len(review_bind))
style['negative']=[0]*len(review_bind)
style['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_style)):
    if(len(adj_style[i]) != 0):
        dictionary=Counter(adj_style[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    style.iat[i,1]=style['positive'][i]+dictionary[word]
                else:
                    style.iat[i,2]=style['negative'][i]+dictionary[word]
        style.iat[i,3]=style['positive'][i]/(style['positive'][i]+style['negative'][i])

style['stars']=review_bind['stars']
style.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,5,0,1.000000,3.5
1,F06m2yQSPHIrb1IT7heYeQ,14,1,0.933333,4.0
2,c_FTil8s5PS2l_YJDQAXhA,0,0,NaN,4.0
3,vAbRNF0hJt91oCUpm_sdKw,0,0,NaN,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,2,0,1.000000,3.5


In [136]:
print(np.sum(style['positive']))
print(np.sum(style['negative']))

1189
15


In [137]:
style.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,-0.047149
stars,-0.047149,1.000000


In [138]:
style.to_csv("style.csv",index=False)

## flavor

In [139]:
adj_flavor=findadj('flavor',clean_text)

In [141]:
flavor=pd.DataFrame(review_bind['business_id'])
flavor['positive']=[0]*(len(review_bind))
flavor['negative']=[0]*len(review_bind)
flavor['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_flavor)):
    if(len(adj_flavor[i]) != 0):
        dictionary=Counter(adj_flavor[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    flavor.iat[i,1]=flavor['positive'][i]+dictionary[word]
                else:
                    flavor.iat[i,2]=flavor['negative'][i]+dictionary[word]
        flavor.iat[i,3]=flavor['positive'][i]/(flavor['positive'][i]+flavor['negative'][i])

flavor['stars']=review_bind['stars']
flavor.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,23,2,0.92,3.5
1,F06m2yQSPHIrb1IT7heYeQ,4,1,0.80,4.0
2,c_FTil8s5PS2l_YJDQAXhA,0,0,NaN,4.0
3,vAbRNF0hJt91oCUpm_sdKw,4,0,1.00,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,6,0,1.00,3.5


In [142]:
print(np.sum(flavor['positive']))
print(np.sum(flavor['negative']))

4228
279


In [143]:
flavor.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.167174
stars,0.167174,1.000000


In [144]:
flavor.to_csv("flavor.csv",index=False)

## plate

In [145]:
adj_plate=findadj('plate',clean_text)

In [147]:
plate=pd.DataFrame(review_bind['business_id'])
plate['positive']=[0]*(len(review_bind))
plate['negative']=[0]*len(review_bind)
plate['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_plate)):
    if(len(adj_plate[i]) != 0):
        dictionary=Counter(adj_plate[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    plate.iat[i,1]=plate['positive'][i]+dictionary[word]
                else:
                    plate.iat[i,2]=plate['negative'][i]+dictionary[word]
        plate.iat[i,3]=plate['positive'][i]/(plate['positive'][i]+plate['negative'][i])

plate['stars']=review_bind['stars']
plate.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,11,1,0.916667,3.5
1,F06m2yQSPHIrb1IT7heYeQ,1,0,1.000000,4.0
2,c_FTil8s5PS2l_YJDQAXhA,0,0,NaN,4.0
3,vAbRNF0hJt91oCUpm_sdKw,1,2,0.333333,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,7,0,1.000000,3.5


In [148]:
print(np.sum(plate['positive']))
print(np.sum(plate['negative']))

1931
325


In [149]:
plate.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.119764
stars,0.119764,1.000000


In [150]:
plate.to_csv("plate.csv",index=False)

## service

In [151]:
adj_service=findadj('service',clean_text)

In [153]:
service=pd.DataFrame(review_bind['business_id'])
service['positive']=[0]*(len(review_bind))
service['negative']=[0]*len(review_bind)
service['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_service)):
    if(len(adj_service[i]) != 0):
        dictionary=Counter(adj_service[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    service.iat[i,1]=service['positive'][i]+dictionary[word]
                else:
                    service.iat[i,2]=service['negative'][i]+dictionary[word]
        service.iat[i,3]=service['positive'][i]/(service['positive'][i]+service['negative'][i])

service['stars']=review_bind['stars']
service.head()

,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,94,21,0.817391,3.5
1,F06m2yQSPHIrb1IT7heYeQ,13,3,0.812500,4.0
2,c_FTil8s5PS2l_YJDQAXhA,13,1,0.928571,4.0
3,vAbRNF0hJt91oCUpm_sdKw,20,7,0.740741,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,34,2,0.944444,3.5


In [154]:
print(np.sum(service['positive']))
print(np.sum(service['negative']))

15458
3162


In [155]:
service.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.580465
stars,0.580465,1.000000


In [156]:
service.to_csv("service.csv",index=False)

## price

In [157]:
adj_price=findadj('price',clean_text)

In [159]:
price=pd.DataFrame(review_bind['business_id'])
price['positive']=[0]*(len(review_bind))
price['negative']=[0]*len(review_bind)
price['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_price)):
    if(len(adj_price[i]) != 0):
        dictionary=Counter(adj_price[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    price.iat[i,1]=price['positive'][i]+dictionary[word]
                else:
                    price.iat[i,2]=price['negative'][i]+dictionary[word]
        price.iat[i,3]=price['positive'][i]/(price['positive'][i]+price['negative'][i])

price['stars']=review_bind['stars']
price.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,32,5,0.864865,3.5
1,F06m2yQSPHIrb1IT7heYeQ,11,0,1.000000,4.0
2,c_FTil8s5PS2l_YJDQAXhA,5,0,1.000000,4.0
3,vAbRNF0hJt91oCUpm_sdKw,9,2,0.818182,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,13,2,0.866667,3.5


In [160]:
print(np.sum(price['positive']))
print(np.sum(price['negative']))

7869
802


In [161]:
price.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.221228
stars,0.221228,1.000000


In [162]:
price.to_csv("price.csv",index=False)

## desert

In [163]:
adj_desert=findadj('desert',clean_text)

In [165]:
desert=pd.DataFrame(review_bind['business_id'])
desert['positive']=[0]*(len(review_bind))
desert['negative']=[0]*len(review_bind)
desert['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_desert)):
    if(len(adj_desert[i]) != 0):
        dictionary=Counter(adj_desert[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    desert.iat[i,1]=desert['positive'][i]+dictionary[word]
                else:
                    desert.iat[i,2]=desert['negative'][i]+dictionary[word]
        desert.iat[i,3]=desert['positive'][i]/(desert['positive'][i]+desert['negative'][i])

desert['stars']=review_bind['stars']
desert.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,8,0,1.0,3.5
1,F06m2yQSPHIrb1IT7heYeQ,0,0,NaN,4.0
2,c_FTil8s5PS2l_YJDQAXhA,0,0,NaN,4.0
3,vAbRNF0hJt91oCUpm_sdKw,2,0,1.0,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,1,0,1.0,3.5


In [166]:
print(np.sum(desert['positive']))
print(np.sum(desert['negative']))

358
13


In [167]:
desert.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.075071
stars,0.075071,1.000000


In [168]:
desert.to_csv("desert.csv",index=False)

## appetizer

In [169]:
adj_appetizer=findadj('appetizer',clean_text)

In [171]:
appetizer=pd.DataFrame(review_bind['business_id'])
appetizer['positive']=[0]*(len(review_bind))
appetizer['negative']=[0]*len(review_bind)
appetizer['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_appetizer)):
    if(len(adj_appetizer[i]) != 0):
        dictionary=Counter(adj_appetizer[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    appetizer.iat[i,1]=appetizer['positive'][i]+dictionary[word]
                else:
                    appetizer.iat[i,2]=appetizer['negative'][i]+dictionary[word]
        appetizer.iat[i,3]=appetizer['positive'][i]/(appetizer['positive'][i]+appetizer['negative'][i])

appetizer['stars']=review_bind['stars']
appetizer.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,23,0,1.000000,3.5
1,F06m2yQSPHIrb1IT7heYeQ,0,0,NaN,4.0
2,c_FTil8s5PS2l_YJDQAXhA,1,0,1.000000,4.0
3,vAbRNF0hJt91oCUpm_sdKw,5,1,0.833333,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,4,0,1.000000,3.5


In [172]:
print(np.sum(appetizer['positive']))
print(np.sum(appetizer['negative']))

1824
92


In [173]:
appetizer.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.057042
stars,0.057042,1.000000


In [174]:
appetizer.to_csv("appetizer.csv",index=False)

## bread

In [177]:
adj_bread=findadj('bread',clean_text)

In [179]:
bread=pd.DataFrame(review_bind['business_id'])
bread['positive']=[0]*(len(review_bind))
bread['negative']=[0]*len(review_bind)
bread['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_bread)):
    if(len(adj_bread[i]) != 0):
        dictionary=Counter(adj_bread[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    bread.iat[i,1]=bread['positive'][i]+dictionary[word]
                else:
                    bread.iat[i,2]=bread['negative'][i]+dictionary[word]
        bread.iat[i,3]=bread['positive'][i]/(bread['positive'][i]+bread['negative'][i])

bread['stars']=review_bind['stars']
bread.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,34,4,0.894737,3.5
1,F06m2yQSPHIrb1IT7heYeQ,0,0,NaN,4.0
2,c_FTil8s5PS2l_YJDQAXhA,1,0,1.000000,4.0
3,vAbRNF0hJt91oCUpm_sdKw,12,1,0.923077,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,11,0,1.000000,3.5


In [180]:
print(np.sum(bread['positive']))
print(np.sum(bread['negative']))

2066
140


In [181]:
bread.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.004778
stars,0.004778,1.000000


In [182]:
bread.to_csv("price.csv",index=False)

## salad

In [183]:
adj_salad=findadj('salad',clean_text)

In [185]:
salad=pd.DataFrame(review_bind['business_id'])
salad['positive']=[0]*(len(review_bind))
salad['negative']=[0]*len(review_bind)
salad['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_salad)):
    if(len(adj_salad[i]) != 0):
        dictionary=Counter(adj_salad[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    salad.iat[i,1]=salad['positive'][i]+dictionary[word]
                else:
                    salad.iat[i,2]=salad['negative'][i]+dictionary[word]
        salad.iat[i,3]=salad['positive'][i]/(salad['positive'][i]+salad['negative'][i])

salad['stars']=review_bind['stars']
salad.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,21,1,0.954545,3.5
1,F06m2yQSPHIrb1IT7heYeQ,0,0,NaN,4.0
2,c_FTil8s5PS2l_YJDQAXhA,1,0,1.000000,4.0
3,vAbRNF0hJt91oCUpm_sdKw,2,0,1.000000,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,2,1,0.666667,3.5


In [186]:
print(np.sum(salad['positive']))
print(np.sum(salad['negative']))

1773
96


In [187]:
salad.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.054725
stars,0.054725,1.000000


In [188]:
salad.to_csv("salad.csv",index=False)

## side

In [189]:
adj_side=findadj('side',clean_text)

In [191]:
side=pd.DataFrame(review_bind['business_id'])
side['positive']=[0]*(len(review_bind))
side['negative']=[0]*len(review_bind)
side['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_side)):
    if(len(adj_side[i]) != 0):
        dictionary=Counter(adj_side[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    side.iat[i,1]=side['positive'][i]+dictionary[word]
                else:
                    side.iat[i,2]=side['negative'][i]+dictionary[word]
        side.iat[i,3]=side['positive'][i]/(side['positive'][i]+side['negative'][i])

side['stars']=review_bind['stars']
side.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,29,5,0.852941,3.5
1,F06m2yQSPHIrb1IT7heYeQ,9,1,0.900000,4.0
2,c_FTil8s5PS2l_YJDQAXhA,1,0,1.000000,4.0
3,vAbRNF0hJt91oCUpm_sdKw,12,1,0.923077,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,10,1,0.909091,3.5


In [192]:
print(np.sum(side['positive']))
print(np.sum(side['negative']))

3711
392


In [193]:
side.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.205917
stars,0.205917,1.000000


In [194]:
side.to_csv("side.csv",index=False)

## soup

In [195]:
adj_soup=findadj('soup',clean_text)

In [197]:
soup=pd.DataFrame(review_bind['business_id'])
soup['positive']=[0]*(len(review_bind))
soup['negative']=[0]*len(review_bind)
soup['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_soup)):
    if(len(adj_soup[i]) != 0):
        dictionary=Counter(adj_soup[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    soup.iat[i,1]=soup['positive'][i]+dictionary[word]
                else:
                    soup.iat[i,2]=soup['negative'][i]+dictionary[word]
        soup.iat[i,3]=soup['positive'][i]/(soup['positive'][i]+soup['negative'][i])

soup['stars']=review_bind['stars']
soup.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,4,0,1.0,3.5
1,F06m2yQSPHIrb1IT7heYeQ,9,0,1.0,4.0
2,c_FTil8s5PS2l_YJDQAXhA,0,0,NaN,4.0
3,vAbRNF0hJt91oCUpm_sdKw,3,0,1.0,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,2,0,1.0,3.5


In [198]:
print(np.sum(soup['positive']))
print(np.sum(soup['negative']))

1555
68


In [199]:
soup.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,-0.012645
stars,-0.012645,1.000000


In [200]:
soup.to_csv("soup.csv",index=False)

## drink

In [201]:
adj_drink=findadj('drink',clean_text)

In [203]:
drink=pd.DataFrame(review_bind['business_id'])
drink['positive']=[0]*(len(review_bind))
drink['negative']=[0]*len(review_bind)
drink['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_drink)):
    if(len(adj_drink[i]) != 0):
        dictionary=Counter(adj_drink[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    drink.iat[i,1]=drink['positive'][i]+dictionary[word]
                else:
                    drink.iat[i,2]=drink['negative'][i]+dictionary[word]
        drink.iat[i,3]=drink['positive'][i]/(drink['positive'][i]+drink['negative'][i])

drink['stars']=review_bind['stars']
drink.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,29,3,0.906250,3.5
1,F06m2yQSPHIrb1IT7heYeQ,0,0,NaN,4.0
2,c_FTil8s5PS2l_YJDQAXhA,3,0,1.000000,4.0
3,vAbRNF0hJt91oCUpm_sdKw,3,1,0.750000,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,2,1,0.666667,3.5


In [204]:
print(np.sum(drink['positive']))
print(np.sum(drink['negative']))

2409
222


In [205]:
drink.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.00000,0.15451
stars,0.15451,1.00000


In [206]:
drink.to_csv("drink.csv",index=False)

## server

In [207]:
adj_server=findadj('server',clean_text)

In [209]:
server=pd.DataFrame(review_bind['business_id'])
server['positive']=[0]*(len(review_bind))
server['negative']=[0]*len(review_bind)
server['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_server)):
    if(len(adj_server[i]) != 0):
        dictionary=Counter(adj_server[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    server.iat[i,1]=server['positive'][i]+dictionary[word]
                else:
                    server.iat[i,2]=server['negative'][i]+dictionary[word]
        server.iat[i,3]=server['positive'][i]/(server['positive'][i]+server['negative'][i])

server['stars']=review_bind['stars']
server.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,15,3,0.833333,3.5
1,F06m2yQSPHIrb1IT7heYeQ,4,0,1.000000,4.0
2,c_FTil8s5PS2l_YJDQAXhA,2,0,1.000000,4.0
3,vAbRNF0hJt91oCUpm_sdKw,3,1,0.750000,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,9,0,1.000000,3.5


In [210]:
print(np.sum(server['positive']))
print(np.sum(server['negative']))

2796
363


In [211]:
server.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.098419
stars,0.098419,1.000000


In [212]:
server.to_csv("server.csv",index=False)

## size

In [213]:
adj_size=findadj('size',clean_text)

In [215]:
size=pd.DataFrame(review_bind['business_id'])
size['positive']=[0]*(len(review_bind))
size['negative']=[0]*len(review_bind)
size['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_size)):
    if(len(adj_size[i]) != 0):
        dictionary=Counter(adj_size[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    size.iat[i,1]=size['positive'][i]+dictionary[word]
                else:
                    size.iat[i,2]=size['negative'][i]+dictionary[word]
        size.iat[i,3]=size['positive'][i]/(size['positive'][i]+size['negative'][i])

size['stars']=review_bind['stars']
size.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,5,3,0.625,3.5
1,F06m2yQSPHIrb1IT7heYeQ,0,1,0.000,4.0
2,c_FTil8s5PS2l_YJDQAXhA,0,0,NaN,4.0
3,vAbRNF0hJt91oCUpm_sdKw,2,0,1.000,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,1,0,1.000,3.5


In [216]:
print(np.sum(size['positive']))
print(np.sum(size['negative']))

1217
137


In [217]:
size.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.000978
stars,0.000978,1.000000


In [218]:
size.to_csv("size.csv",index=False)

## food

In [219]:
adj_food=findadj('food',clean_text)

In [220]:
print(adj_food)

[['u', 'birthday', 'good', 'delicious', 'excellent', 'long', 'amazing', 'dry', 'cellar', 'attentive', 'bad', 'bad', 'underquality', 'cellar', 'average', 'eclectic', 'memorable', 'wonderful', 'different', 'dent', 'romantic', 'great', 'upscale', 'good', 'bad', 'good', 'much', 'fabulous', 'attentive', 'good', 'much', 'anniversary', 'great', 'disappointed', 'sloppy', 'incredible', 'delicious', 'wonderful', 'eat', 'important', 'awesome', 'rich', 'good', 'good', 'enjoy', 'helpful', 'good', 'comfortable', 'great', 'bad', 'good', 'great', 'helpful', 'steak', 'mediocre', 'entree', 'fresh', 'excessive', 'fresh', 'good', 'nice', 'great', 'memorable', 'great', 'delight', 'terrible', 'quality', 'delicious', 'cellar', 'excitement', 'negative', 'great', 'great', 'great', 'table', 'good', 'overall', 'phoenix', 'goodness', 'knowledgeable', 'great', 'bad', 'bad', 'much', 'amazing', 'amazing', 'not_ordering', 'great', 'enjoyable', 'not_private', 'wildfish', 'awesome', 'good', 'good', 'ordinary', 'good', 

In [221]:
food=pd.DataFrame(review_bind['business_id'])
food['positive']=[0]*(len(review_bind))
food['negative']=[0]*len(review_bind)
food['proportion']=[np.nan]*len(review_bind)

from collections import Counter

for i in range(len(adj_food)):
    if(len(adj_food[i]) != 0):
        dictionary=Counter(adj_food[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    food.iat[i,1]=food['positive'][i]+dictionary[word]
                else:
                    food.iat[i,2]=food['negative'][i]+dictionary[word]
        food.iat[i,3]=food['positive'][i]/(food['positive'][i]+food['negative'][i])

food['stars']=review_bind['stars']
food.head()

,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,102,15,0.871795,3.5
1,F06m2yQSPHIrb1IT7heYeQ,70,1,0.985915,4.0
2,c_FTil8s5PS2l_YJDQAXhA,9,0,1.000000,4.0
3,vAbRNF0hJt91oCUpm_sdKw,13,1,0.928571,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,54,4,0.931034,3.5


In [222]:
print(np.sum(food['positive']))
print(np.sum(food['negative']))

24047
1907


In [223]:
food.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.547101
stars,0.547101,1.000000


In [224]:
food.to_csv("size.csv",index=False)